Set up environment and load in data

In [ ]:
library(tidyverse)
library(here)
library(grid)
library(gridExtra)
theme_set(theme_bw())
helpers_path = paste0(here(),'/analysis/helpers/')
source(paste0(helpers_path, '01_clean_behavioral_data.R'))
rm(data_bc_clean)
fig_out_path = paste0(here(), '/outputs/fig/')

**Make sure to scale value and exclude too fast trials before fitting**

In [ ]:
normMax = 1
normMin = -1

data_yn_clean = data_yn_clean %>%
  filter(reference != -99) %>%
  filter(rt > .3 & rt < 5) %>% # discard very long and short RT trials
  group_by(subnum, day, type) %>%
  mutate(possiblePayoff_dmn = possiblePayoff - mean(possiblePayoff)) %>%
  mutate(rawVDiff = possiblePayoff - reference,
         normVDiff =  (normMax - normMin) / (max(rawVDiff) - min(rawVDiff)) * (rawVDiff - max(rawVDiff)) + (normMax) )

rm(normMax, normMin)

# Exploring priors and parameter space

What does a subject prior for drift rates look like in the HDDM (from Lombardi-Hare toolbox)

```
d.mu ~ dunif(.00001, 50)
d.pr ~ dgamma(1, .1)
d[subject] ~ dnorm(d.mu, d.pr) [trimmed at 10e-5, 50]

```

In [ ]:
n = 10000

dat = data.frame(d.mu = runif(n, .00001, 50), d.pr = rgamma(n, 1, .1))

for(i in 1:nrow(dat)){
  dat$d[i] = rnorm(1, dat$d.mu[i], dat$d.pr[i])
}

dat$d = ifelse(dat$d > 50, 50, ifelse(dat$d<.000001, .000001, dat$d))

dat %>%
  # select(d) %>%
  gather(key, value) %>%
  # mutate(key = factor(key, levels = c("d.pr", "d.mu", "d"))) %>%
  ggplot(aes(value, fill=key))+
  geom_histogram(position = "identity", alpha = .5, bins = 30)+
  facet_wrap(~key, scales = "free")+
  theme(legend.position = "bottom")+
  labs(fill="", x="")

What about the bias and ndt? Uniform-ish too.

```
bias[p] ~ dbeta(bias.alpha, bias.beta)T(0.01,0.99)

bias.alpha <- bias.mu * bias.kappa
bias.beta <- (1 - bias.mu) * bias.kappa
bias.mu ~ dbeta(2, 2)T(0.01,0.99)
bias.kappa ~ dgamma(1, 0.5)
```

**NOTE: This is operationalized in the HDDM, ranging from 0 to 1 and no bias is .5**

In [ ]:
n = 10000

dat = data.frame(bias.kappa = rgamma(n, 1, .5), bias.mu = rbeta(n, 2, 2))

dat$bias.mu = ifelse(dat$bias.mu < .01, .01, ifelse(dat$bias.mu > .99, .99, dat$bias.mu))

dat = dat %>%
  mutate(bias.beta = (1 - bias.mu) * bias.kappa,
         bias.alpha = bias.mu * bias.kappa)

for(i in 1:nrow(dat)){
  dat$bias[i] = rbeta(1, dat$bias.alpha[i], dat$bias.beta[i])
}

dat$bias = ifelse(dat$bias < .01, .01, ifelse(dat$bias > .99, .99, dat$bias))

dat %>%
  gather(key, value) %>%
  ggplot(aes(value, fill=key))+
  geom_histogram(position = "identity", alpha = .5, bins = 30)+
  facet_wrap(~key, scales = "free")+
  theme(legend.position = "bottom")+
  labs(fill="", x="")


```
theta.p[p] ~ dnorm(ndt.mu, ndt.pr)T(0.01, 1)
ndt.pr ~ dgamma(1, 0.1)
ndt.mu ~ dunif(0, 1)
```

In [ ]:
n = 10000

dat = data.frame(ndt.mu = runif(n, 0, 1), ndt.pr = rgamma(n, 1, .1))

for(i in 1:nrow(dat)){
  dat$ndt[i] = rnorm(1, dat$ndt.mu[i], dat$ndt.pr[i])
}

dat$ndt = ifelse(dat$ndt < .01, .01, ifelse(dat$ndt > 1, 1, dat$ndt))

dat %>%
  gather(key, value) %>%
  ggplot(aes(value, fill=key))+
  geom_histogram(position = "identity", alpha = .5, bins = 30)+
  facet_wrap(~key, scales = "free")+
  theme(legend.position = "bottom")+
  labs(fill="", x="")


What are reasonable values for barrierDecay? Prior: Uniform[0, .01]; min = 0, max = 0.02

In [ ]:
decay_vals = c(0, .001, .01, .02, .1, 1)

maxIter = 400

dat = data.frame()

for(cur_dv in decay_vals){
  cur_dat = data.frame(t = c(0:maxIter),
                       initialBarrier = 1, 
                       barrierDecay = cur_dv)
  
  cur_dat = cur_dat %>%
    mutate(barrier = initialBarrier / (1 + (barrierDecay * t)))
  
  dat = rbind(dat, cur_dat)
}

dat %>%
  mutate(barrierDecay = factor(barrierDecay, levels = decay_vals)) %>%
  ggplot(aes(t, barrier, color = barrierDecay))+
  geom_point()+
  theme_bw()+
  theme(legend.position = "bottom")

In [ ]:
decay_vals = c(0, .001, .01, .02, .1, .2, .5, 1)

maxIter = 500

dat = data.frame()

for(cur_dv in decay_vals){
  cur_dat = data.frame(t = c(0:maxIter),
                       initialBarrier = 2, 
                       barrierDecay = cur_dv)
  
  cur_dat = cur_dat %>%
    mutate(barrier = initialBarrier / (1 + (barrierDecay * t)))
  
  dat = rbind(dat, cur_dat)
}

dat %>%
  mutate(barrierDecay = factor(barrierDecay, levels = decay_vals)) %>%
  ggplot(aes(t, barrier, color = barrierDecay))+
  geom_point()+
  theme_bw()+
  theme(legend.position = "bottom")

In [ ]:
dat %>%
  filter(barrierDecay == .5) %>%
  arrange(-t)

In [ ]:
n = 10000

dat = data.frame(decay.alpha = rbeta(n, .1, 1), decay.beta = rgamma(n, 2, .5))
dat$decay.alpha = ifelse(dat$decay.alpha < 0, 0, ifelse(dat$decay.alpha > .5, .5, dat$decay.alpha))

for(i in 1:nrow(dat)){
  dat$decay[i] = rbeta(1, dat$decay.alpha[i], dat$decay.beta[i])
}

dat$decay = ifelse(dat$decay < 0, 0, ifelse(dat$decay > .5, .5, dat$decay))

dat %>%
  gather(key, value) %>%
  ggplot(aes(value, fill=key))+
  geom_histogram(position = "identity", alpha = .5, bins = 30)+
  facet_wrap(~key, scales = "free")+
  theme(legend.position = "bottom")+
  labs(fill="", x="")


# Test trial simulation function

Simulate a few trials

In [ ]:
source(paste0(helpers_path, '/ddm/yn_ddm.R'))

In [ ]:
cur_d = .03
cur_sigma = .02
cur_nonDecisionTime = 300 #ms
cur_bias = 0
cur_barrierDecay = .002

cur_valStim = .25
cur_valRef = 0
cur_valDiff = cur_valStim - cur_valRef

tmp = sim_trial(d = cur_d, sigma = cur_sigma, nonDecisionTime = cur_nonDecisionTime, bias = cur_bias, barrierDecay = cur_barrierDecay, ValDiff = cur_valDiff, debug = T)

tmp

In [ ]:
tmp$debug_df %>%
  ggplot()+
  geom_line(aes(time, RDV), color = "#F8766D", size = 2)+
  geom_line(aes(time, barrier))+
  geom_line(aes(time, -barrier))+
  theme(panel.grid = element_blank())+
  geom_hline(aes(yintercept = 0), linetype = "dashed")+
  geom_hline(aes(yintercept = 1), linetype = "dashed")+
  geom_hline(aes(yintercept = -1), linetype = "dashed")+
  geom_vline(aes(xintercept = 30), color = "gray")+
  labs(x = "Timestep (in 10 ms)")

# Test task simulation function

In [ ]:
source(paste0(helpers_path, '/ddm/sim_yn_ddm.R'))

In [ ]:
cur_d = .08
cur_sigma = .03
cur_nonDecisionTime = 200 #ms
cur_bias = 0
cur_barrierDecay = .008

sub_stims = data_yn_clean %>%
  filter((subnum == 611) & (day == 4) & (type == 1)) %>%
  filter(reference != -99) %>%
  filter(rt > .3 & rt < 2)

# names(sub_stims)

sim_trial_list = list("model1" = sim_trial)

tmp = sim_task(stimuli = sub_stims, model_name = "model1", sim_trial_list_ = sim_trial_list, d = cur_d, sigma = cur_sigma, nonDecisionTime = cur_nonDecisionTime, bias = cur_bias, barrierDecay = cur_barrierDecay, maxIter = 300, debug = T)

tmp



Are these reasonable parameter values?

In [ ]:
true_dat = sub_stims %>%
  ungroup() %>%
  select(yesChosen, rt) %>%
  mutate(rt = ifelse(yesChosen == 0, (-1) * rt, rt),
         data = "true")

sim_dat = tmp %>%
  select(choice, reactionTime) %>%
  mutate(yesChosen = ifelse(choice == "yes", 1, 0),
         rt = ifelse(choice == "yes", reactionTime, (-1) * reactionTime),
         data = "sim") %>%
  select(-choice, -reactionTime)

ggplot()+
  geom_histogram(data = true_dat, aes(rt, y = ..density..), bins = 30, alpha = 0.5)+
  geom_density(data = sim_dat, aes(rt))

# Test trial likelihood function

In [ ]:
source(paste0(helpers_path, '/ddm/yn_ddm.R'))

In [ ]:
i = 25

tmp[i,]

In [ ]:
fit_trial(d = cur_d, sigma = cur_sigma, nonDecisionTime = cur_nonDecisionTime, bias = cur_bias, barrierDecay = cur_barrierDecay, choice = tmp$choice[i], reactionTime = tmp$reactionTime[i], ValDiff = tmp$ValDiff[i], approxStateStep = .1)

In [ ]:
fit_trial(d = cur_d, sigma = cur_sigma, nonDecisionTime = cur_nonDecisionTime, bias = cur_bias, barrierDecay = cur_barrierDecay, choice = tmp$choice[i], reactionTime = tmp$reactionTime[i], ValDiff = tmp$ValDiff[i], approxStateStep = .01)

**Note higher (and incorrect!) parameter has higher likelihood with large state step size!**

In [ ]:
fit_trial(d = cur_d*10, sigma = cur_sigma*10, nonDecisionTime = cur_nonDecisionTime, bias = cur_bias, barrierDecay = cur_barrierDecay, choice = tmp$choice[i], reactionTime = tmp$reactionTime[i], ValDiff = tmp$ValDiff[i], approxStateStep = .1)

In [ ]:
fit_trial(d = cur_d*10, sigma = cur_sigma*10, nonDecisionTime = cur_nonDecisionTime, bias = cur_bias, barrierDecay = cur_barrierDecay, choice = tmp$choice[i], reactionTime = tmp$reactionTime[i], ValDiff = tmp$ValDiff[i], approxStateStep = .01)

In [ ]:
fit_trial(d = cur_d/2, sigma = cur_sigma, nonDecisionTime = cur_nonDecisionTime, bias = cur_bias, barrierDecay = cur_barrierDecay, choice = tmp$choice[i], reactionTime = tmp$reactionTime[i], ValDiff = tmp$ValDiff[i], approxStateStep = .01)

## Debugging state space

In [ ]:
source(paste0(helpers_path, '/ddm/yn_ddm.R'))

In [ ]:
tmp = sim_trial(d = .005, sigma = .07, nonDecisionTime = 200, bias = 0, barrierDecay = 0.001, ValDiff = -1, debug = T)

tmp$debug_df %>%
  ggplot()+
  geom_line(aes(time, RDV), color = "#F8766D", size = 2)+
  geom_line(aes(time, barrier))+
  geom_line(aes(time, -barrier))+
  theme(panel.grid = element_blank())+
  geom_hline(aes(yintercept = 0), linetype = "dashed")+
  geom_hline(aes(yintercept = 1), linetype = "dashed")+
  geom_hline(aes(yintercept = -1), linetype = "dashed")+
  labs(x = "Timestep (in 10 ms)")

In [ ]:
sim_trial_list = list("model1" = sim_trial)

tmp = sim_task(stimuli = tibble(normVDiff = rep(-1, 200)), model_name = "model1", sim_trial_list_ = sim_trial_list, d = .005, sigma = .07, nonDecisionTime = 200, bias = 0, barrierDecay = 0.001)

summary(tmp$reactionTime)

In [ ]:
debug_out = fit_trial(d = .005, sigma = .07, nonDecisionTime = 200, bias = 0, barrierDecay = 0, ValDiff = -1, choice = -1, reactionTime = 1.5, debug=T)

str(debug_out)

In [ ]:
annotations <- data.frame(
        xpos = c(-Inf),
        ypos =  c(Inf),
        annotateText = c(paste0("Choice: ", debug_out$likelihood$choice,
                         "\nRT: ", debug_out$likelihood$reactionTime,
                         "\nValDiff: ", debug_out$likelihood$ValDiff,
                         "\nd: ", debug_out$likelihood$d,
                         "\nsigma: ", debug_out$likelihood$sigma,
                         "\nndt: ", debug_out$likelihood$nonDecisionTime,
                         "\nlik: ", round(debug_out$likelihood$likelihood, 6))),
        hjustvar = c(-0.1) ,
        vjustvar = c(1.1))

tibble(probUpCrossing = debug_out$probUpCrossing,
       probDownCrossing = debug_out$probDownCrossing) %>%
  mutate(time = 1:n() * (debug_out$likelihood$timeStep)) %>%
  gather(key, value, -time) %>%
  ggplot(aes(time, value))+
  geom_line( aes(color = key) )+
  labs(y = "likelihood", color = "") +
  theme(legend.position = "bottom") +
  geom_text(data = annotations, aes(x = xpos, y = ypos, 
                                    hjust = hjustvar,
                                    vjust = vjustvar,
                                    label = annotateText))

In [ ]:
debug_out$renorm_debug %>%
  select(sumIn, cumSumPUp, cumSumPDown) %>%
  mutate(total = sumIn + cumSumPUp + cumSumPDown,
         time = 1:n() * debug_out$likelihood$timeStep) %>%
  gather(key, value, -time) %>%
  ggplot(aes(time, value))+
  geom_line( aes(color = key) )+
  labs(y = "Cumulative prob", color = "") +
  theme(legend.position = "bottom") +
  geom_text(data = annotations, aes(x = xpos, y = ypos, 
                                    hjust = hjustvar,
                                    vjust = vjustvar,
                                    label = annotateText))


What are these kinks in the distributions when there is decay in barriers?

In [ ]:
debug_out = fit_trial(d = .005, sigma = .07, nonDecisionTime = 200, bias = 0, barrierDecay = 0.001, ValDiff = -1, choice = -1, reactionTime = 1.5, debug=T)

annotations <- data.frame(
        xpos = c(-Inf),
        ypos =  c(Inf),
        annotateText = c(paste0("Choice: ", debug_out$likelihood$choice,
                         "\nRT: ", debug_out$likelihood$reactionTime,
                         "\nValDiff: ", debug_out$likelihood$ValDiff,
                         "\nd: ", debug_out$likelihood$d,
                         "\nsigma: ", debug_out$likelihood$sigma,
                         "\nndt: ", debug_out$likelihood$nonDecisionTime)),
        hjustvar = c(-0.1) ,
        vjustvar = c(1.1))

tibble(probUpCrossing = debug_out$probUpCrossing,
       probDownCrossing = debug_out$probDownCrossing) %>%
  mutate(time = 1:n() * (debug_out$likelihood$timeStep)) %>%
  gather(key, value, -time) %>%
  ggplot(aes(time, value))+
  geom_line( aes(color = key) )+
  labs(y = "likelihood", color = "") +
  theme(legend.position = "bottom") +
  geom_text(data = annotations, aes(x = xpos, y = ypos, 
                                    hjust = hjustvar,
                                    vjust = vjustvar,
                                    label = annotateText))

debug_out$renorm_debug %>%
  select(sumIn, cumSumPUp, cumSumPDown) %>%
  mutate(total = sumIn + cumSumPUp + cumSumPDown,
         time = 1:n() * debug_out$likelihood$timeStep) %>%
  gather(key, value, -time) %>%
  ggplot(aes(time, value))+
  geom_line( aes(color = key) )+
  labs(y = "Cumulative prob", color = "") +
  theme(legend.position = "bottom") +
  geom_text(data = annotations, aes(x = xpos, y = ypos, 
                                    hjust = hjustvar,
                                    vjust = vjustvar,
                                    label = annotateText))

In [ ]:
debug_out = fit_trial(d = .005, sigma = .07, nonDecisionTime = 200, bias = 0, barrierDecay = 0.001, ValDiff= -1, choice = -1, reactionTime = 3.5, debug=T)

annotations <- data.frame(
        xpos = c(-Inf),
        ypos =  c(Inf),
        annotateText = c(paste0("Choice: ", debug_out$likelihood$choice,
                         "\nRT: ", debug_out$likelihood$reactionTime,
                         "\nValDiff: ", debug_out$likelihood$ValDiff,
                         "\nd: ", debug_out$likelihood$d,
                         "\nsigma: ", debug_out$likelihood$sigma,
                         "\nndt: ", debug_out$likelihood$nonDecisionTime)),
        hjustvar = c(-0.1) ,
        vjustvar = c(1.1))

tibble(probUpCrossing = debug_out$probUpCrossing,
       probDownCrossing = debug_out$probDownCrossing) %>%
  mutate(time = 1:n() * (debug_out$likelihood$timeStep)) %>%
  gather(key, value, -time) %>%
  ggplot(aes(time, value))+
  geom_line( aes(color = key) )+
  labs(y = "likelihood", color = "") +
  theme(legend.position = "bottom") +
  geom_text(data = annotations, aes(x = xpos, y = ypos, 
                                    hjust = hjustvar,
                                    vjust = vjustvar,
                                    label = annotateText))

debug_out$renorm_debug %>%
  select(sumIn, cumSumPUp, cumSumPDown) %>%
  mutate(total = sumIn + cumSumPUp + cumSumPDown,
         time = 1:n() * debug_out$likelihood$timeStep) %>%
  gather(key, value, -time) %>%
  ggplot(aes(time, value))+
  geom_line( aes(color = key) )+
  labs(y = "Cumulative prob", color = "") +
  theme(legend.position = "bottom") +
  geom_text(data = annotations, aes(x = xpos, y = ypos, 
                                    hjust = hjustvar,
                                    vjust = vjustvar,
                                    label = annotateText))

# Test task fitting functions

In [ ]:
source(paste0(helpers_path, '/ddm/fit_yn_ddm.R'))

In [ ]:
cur_d = .03
cur_sigma = .02
cur_nonDecisionTime = 300 #ms
cur_bias = 0
cur_barrierDecay = .002

In [ ]:
sub_stims = data_yn_clean %>%
  filter((subnum == 611) & (day == 4) & (type == 1))

# names(sub_stims)

sim_trial_list = list("model1" = sim_trial)

tmp = sim_task(stimuli = sub_stims, model_name = "model1", sim_trial_list_ = sim_trial_list, d = cur_d, sigma = cur_sigma, nonDecisionTime = cur_nonDecisionTime, bias = cur_bias, barrierDecay = cur_barrierDecay, debug = T)

tmp

In [ ]:
fit_trial_list = list("model1" = fit_trial)

fit_task(data_ = tmp[1:100,], model_name_ = "model1", pars_ = list(d = cur_d, sigma = cur_sigma, nonDecisionTime = cur_nonDecisionTime, bias = cur_bias, barrierDecay = cur_barrierDecay) , fit_trial_list_ = fit_trial_list, debug=FALSE)

Is negative log likelihood lower for the true parameter combination compared to other combinations?

In [ ]:
get_task_nll(data_ = tmp[1:100,], par_ = c(cur_d, cur_sigma, cur_nonDecisionTime, cur_bias, cur_barrierDecay), par_names_ = c("d", "sigma", "nonDecisionTime", "bias", "barrierDecay"), model_name_ = "model1")

In [ ]:
get_task_nll(data_ = tmp[1:100,], par_ = c(cur_d/2, cur_sigma, cur_nonDecisionTime, cur_bias, cur_barrierDecay), par_names_ = c("d", "sigma", "nonDecisionTime", "bias", "barrierDecay"), model_name_ = "model1")

In [ ]:
get_task_nll(data_ = tmp[1:100,], par_ = c(cur_d, cur_sigma/2, cur_nonDecisionTime, cur_bias, cur_barrierDecay), par_names_ = c("d", "sigma", "nonDecisionTime", "bias", "barrierDecay"), model_name_ = "model1")

## Test optim on simulated data

In [ ]:
cur_d = .03
cur_sigma = .02
cur_nonDecisionTime = 300 #ms
cur_bias = 0
cur_barrierDecay = .002

In [ ]:
sub_stims = data_yn_clean %>%
  filter((subnum == 611) & (day == 4) & (type == 1))

sim_trial_list = list("model1" = sim_trial)

tmp = sim_task(stimuli = sub_stims, model_name = "model1", sim_trial_list_ = sim_trial_list, d = cur_d, sigma = cur_sigma, nonDecisionTime = cur_nonDecisionTime, bias = cur_bias, barrierDecay = cur_barrierDecay, debug = T)

In [ ]:
start_vals = c(.01, .01, 100, 0, .001)
par_names = c("d", "sigma", "nonDecisionTime", "bias", "barrierDecay")
model = 'model1'

optim_out = optim(par = start_vals, get_task_nll, data_= tmp, par_names_ = par_names, model_name_ = model, control = list(maxit = 5, trace = 2))

cur_out = tibble(key = par_names, value = optim_out$par)
cur_out = cur_out %>% spread(key, value)
cur_out$nll = optim_out$value

cur_out

## Test optim on single subject

# Examine optim output for a single subject

Read in the estimates for a single subject:

In [ ]:
cur_sub = 601

all_files = list.files(path = paste0(here(), '/inputs/'), pattern= paste0("optim_YN_DDM_FIT_sub-",cur_sub))

optim_out= data.frame()

for (cur_file in all_files){
  cur_in = read.csv(paste0(here(), '/inputs/', cur_file))
  optim_out = rbind(optim_out, cur_in)
}

optim_out

Distribution of estimates

In [ ]:
optim_out %>%
  select(d, sigma, nonDecisionTime, bias, barrierDecay) %>%
  gather(key, value) %>%
  ggplot(aes(value, fill = key))+
  geom_histogram(bins = 30, alpha = .5)+
  facet_wrap(~key, scales = "free")+
  theme(legend.position = "none",
        panel.grid = element_blank())

In [ ]:
p = optim_out %>%
  select(d, sigma, nonDecisionTime, bias, barrierDecay, day, type) %>%
  gather(key, value, -day, -type) %>%
  group_by(key, day, type) %>%
  summarise(mean_par = mean(value), 
            sem_par = sd(value)/sqrt(n()), .groups = "keep") %>%
  mutate(type = ifelse(type == 1, "HT", "RE"),
         key = factor(key, levels = c("d", "sigma", "nonDecisionTime", "bias", "barrierDecay"))) %>%
  ggplot(aes(day, mean_par, color = type))+
  geom_point(position = position_dodge(width=.5))+
  geom_errorbar(aes(ymin = mean_par - sem_par, ymax = mean_par + sem_par), width = .1, position = position_dodge(width=.5)) +
  facet_wrap(~key, scales = "free", ncol = 5) +
  theme(legend.position = "bottom",
        panel.grid.minor = element_blank())+
  scale_color_brewer(palette = "Dark2")+
  labs(y = "", color = "")+
  scale_x_continuous(breaks = seq(1:11))

ggsave(file=paste0(fig_out_path, 'yn_sub-',cur_sub,'_optimDDMparsOverDays.jpg'), p, height = 4, width=12, units="in")


Correlation between estimates

In [ ]:
# Average correlation across days of HT

cor_mats = list()
for(cur_day in c(1:11)){
  cor_dt = optim_out %>%
    filter((type == 1) & (day == cur_day)) %>%
    select(d, sigma, nonDecisionTime, bias, barrierDecay)
  
  cor_mats[[cur_day]] = cor(cor_dt)
  
  
}

mean_cor_mat = Reduce("+", cor_mats) / length(cor_mats)
tg = tableGrob(round(mean_cor_mat, 3) )
ggsave(paste0(fig_out_path, "sub-",cur_sub, "_ddmOptimHTparCors.jpg"), tg, width = 12, height = 4)

In [ ]:
# Average correlation across days of HT

cor_mats = list()
for(cur_day in c(1:11)){
  cor_dt = optim_out %>%
    filter((type == 0) & (day == cur_day)) %>%
    select(d, sigma, nonDecisionTime, bias, barrierDecay)
  
  cor_mats[[cur_day]] = cor(cor_dt)
  
  
}

mean_cor_mat = Reduce("+", cor_mats) / length(cor_mats)
tg = tableGrob(round(mean_cor_mat, 3) )
ggsave(paste0(fig_out_path, "sub-",cur_sub, "_ddmOptimREparCors.jpg"), tg, width = 12, height = 4)

Correlation between each parameter and its starting point

In [ ]:
optim_out %>%
  ggplot(aes(start_d, d))+
  geom_point(size = 1)

optim_out %>%
  ggplot(aes(start_sigma, sigma))+
  geom_point(size = 1)

optim_out %>%
  ggplot(aes(start_nonDecisionTime, nonDecisionTime))+
  geom_point(size = 1)

optim_out %>%
  ggplot(aes(start_bias, bias))+
  geom_point(size = 1)

optim_out %>%
  ggplot(aes(start_barrierDecay, barrierDecay))+
  geom_point(size = 1)

In [ ]:
optim_out %>%
  select(d, start_d)

optim_out %>%
  select(sigma, start_sigma)

optim_out %>%
  select(nonDecisionTime, start_nonDecisionTime)

optim_out %>%
  select(bias, start_bias)

optim_out %>%
  select(barrierDecay, start_barrierDecay)

In [ ]:
optim_out %>%
  mutate(diff_d = abs(d - start_d),
         diff_sigma = abs(sigma - start_sigma),
         diff_bias = abs(bias - start_bias),
         diff_nonDecisionTime = abs(nonDecisionTime - start_nonDecisionTime),
         diff_barrierDecay = abs(barrierDecay - start_barrierDecay)) %>%
  select(diff_d, diff_sigma, diff_bias, diff_nonDecisionTime, diff_barrierDecay) %>%
  gather(key, value) %>%
  select(value) %>%
  distinct()

Posterior predictive data

In [ ]:
source(paste0(helpers_path, '/ddm/yn_ddm.R'))

Checks of stylized behavioral effects